In [16]:

import cv2
from random import shuffle
from skimage.transform import resize
from os import listdir, walk
from os.path import isfile, join
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import random
import pickle
import itertools
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import backend as K


from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam



- From Big Data mushrooms we have two lists, splitted in test and validation.
- - 1604 kinds of fungis
- - 266344 / 19594 , test/ val


- Smalll data set
- - Categories : 'Agaricus' : 0,'Amanita' : 1,'Boletus' : 2,'Cortinarius' : 3,'Entoloma' : 4,'Hygrocybe' : 5,
    'Lactarius' : 6,'Russula' : 7,'Suillus' : 8
- - number of pics per category: [352, 749, 1072, 835, 363, 315, 1562, 1147, 310]

# Day-2
- Work on the small data set.
- - make batches of downsize files
- - see issue with the premature end of JPEG
- - - make model run with 300 img per category
- - - make model work for 1000 pics per category using augmentation.Keras

In [5]:
def getList(dict):
    """
    This function returns the list of keys in a dictionary
    """
    return dict.keys()


CLASSES = {'Agaricus' : 0,'Amanita' : 1,'Boletus' : 2,'Cortinarius' : 3,'Entoloma' : 4,'Hygrocybe' : 5,
    'Lactarius' : 6,'Russula' : 7,'Suillus' : 8
                }

def get_label(key ,class_mush = CLASSES):
    """
    You can get the labeled name of the mushroom by giving 
    the index or the index by giving the key
    """
    if type(key) == str:
        return class_mush[key]
    elif type(key) == int:
        return list(CLASSES.keys())[list(CLASSES.values()).index(key)]


def upload_number_dir_labels(directory, mapping =CLASSES, num = 1000):
    """
    give the list of photos from directory and the labels.
    num = number of pictures I want to take from each FOLDER. 
    """
    label = []
    dir= []
    list_of_mushrooms = getList(mapping)
    for f in list_of_mushrooms:
        pictures_dir = directory + f + '/'
        pictures = [pics for pics in listdir(pictures_dir) if isfile(join(pictures_dir , pics))]
        if np.size(pictures) < num:
            for idx, picture in enumerate(pictures[:num]): # take the first 300 pictures from each category
                label.append(get_label(f))  
                dir.append(pictures_dir + picture) 
        elif np.size(pictures) >= num:
            shuffle(pictures)
            for idx, picture in enumerate(pictures[:num]): # take the first 300 pictures from each category
                label.append(get_label(f))  
                dir.append(pictures_dir + picture)             
        
    return dir, label 


def image_reshape(dir, batch, new_size = (224,224),k = 300, gray =False):
    """
    give the list of photos with the correct pixel size you want to 
    downsize. If you want Gray then write True.
    """
    items_resized = []
    for idx, dir in enumerate(dir[(batch-1)*k:batch * k]):     
        item = cv2.imread(dir)
        if gray == True:
            item_gray = cv2.cvtColor(item, cv2.COLOR_BGR2GRAY)
            item_resized_gray = resize(item_gray, new_size)
            items_resized.append(item_resized_gray)
        else:
            item_resized_colored = resize(item, new_size)
            items_resized.append(item_resized_colored)
    with open(dir + "mushrooms_resized_part_"+ str(batch), "wb") as fp:   #Pickling
        pickle.dump(items_resized, fp)
    return items_resized
    
def pickle_array(item, directory, name):
    """
    THis function pickles an array on a specific directory
    """
    with open(directory + name, "wb") as fp:   #Pickling
        pickle.dump(item, fp)

def downsize_in_batches(directories):
    """
    This function downsizes the images in batches of 300pics.
    directories = list of the directories of the images.
    """
    batch = 1
    for idx, dir in enumerate(directories):

        if idx % 300 == 0:
            print(batch)
            image_reshape(directories,batch)
            batch += 1 


def unpickle(directory , mapping, labels):
    """
    This functions takes the data from a directory that have been
    downsized in banches ang gives back the X,y as arrays.
    """
    
    table_mushrooms_downsized = []
    #directories, labels = upload_dir_labels(directory,mapping, k=300)
    for batch in range(1,10): # 10 is the number of batches
        with open(directory + str(batch), "rb") as fp:   # Unpickling
            li = pickle.load(fp)
            table_mushrooms_downsized.append(li)

    # flatten the nested list into a 1D list
    mushrooms_downsized = list(itertools.chain(*table_mushrooms_downsized))
            
    X = np.asarray(mushrooms_downsized)
    y = np.asarray(labels)

    return X, y

def print_num_pix_per_category(label):
    """
    This function prints the number of pictures per category of mushrooms
    label = list of numbers 0- len(classes)
    """
    res = list(set(label))
    for value in list(set(label)):
        print(f"There are {label.count(value)} values in the mushroom category {get_label(value)}")



In [34]:
directory = '/kaggle/input/' 
# Get the directories of 1000 images per category of mushroom. When is more take random.
dir, label = upload_number_dir_labels(directory = directory, num = 1000)
# Print the number of images per category to help you with augmentation
print_num_pix_per_category(label)

In [ ]:
downsize_in_batches(dir)